# Joining zonal data
We split up into gap codes to compute zonal stats, so we need to join them into a single table again

In [ ]:
import ibis 
from ibis import _
import os
import sys
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if base_dir not in sys.path:
    sys.path.insert(0, base_dir)
    
from minio_utils import * 
con, _ = connect_minio()

In [ ]:
labels = ['gap1','gap2','gap3','gap4','nonconserved']

for label in labels:
    names = ['id',
             'pct_top_amphibian_richness','mean_amphibian_richness',
             'pct_top_reptile_richness','mean_reptile_richness',
             'pct_top_bird_richness','mean_bird_richness',
             'pct_top_mammal_richness','mean_mammal_richness',
             'pct_top_freshwater_richness','mean_top_freshwater_richness',
             'pct_wetlands','pct_fire','pct_farmland','pct_grazing',
             'pct_disadvantaged_community','pct_low_income_community',
             'mean_plant_richness','pct_top_plant_richness'
            ]
    
    agg_dict = {
        name: _[name].first() for name in names
    }
    stats_url = f's3://public-ca30x30/CA_Nature/2024/Preprocessing/v3/stats/{label}/**.parquet'
    a = (con.read_parquet(stats_url, union_by_name = True)
        .drop('geom')
        .group_by('sub_id')
        .aggregate(**agg_dict)
        )
    
    url = f's3://public-ca30x30/CA_Nature/2024/Preprocessing/v3/subsets/split_habitat_climate/{label}_habitat_climate.parquet'
    base = con.read_parquet(url)
    joined = base.inner_join(a,['sub_id','id'])
    save_url = f's3://public-ca30x30/CA_Nature/2024/Preprocessing/v3/stats/{label}_habitat_climate_stats.parquet'
    joined.to_parquet(save_url)

In [ ]:
cols = ['id',
    'sub_id',
    'name',
    'manager',
    'manager_type',
    'gap_code',
    'status',
    'land_tenure',
    'access_type',
    'county',
    'ecoregion',
    'habitat_type',
    'climate_zone',
    'mean_amphibian_richness',
    'mean_reptile_richness',
    'mean_bird_richness',
    'mean_mammal_richness',
    'mean_plant_richness',
    'mean_freshwater_richness',
    'pct_top_amphibian_richness',
    'pct_top_reptile_richness',
    'pct_top_bird_richness',
    'pct_top_mammal_richness',
    'pct_top_plant_richness',
    'pct_top_freshwater_richness',
    'pct_wetlands',
    'pct_fire',
    'pct_farmland',
    'pct_grazing_lands',
    'pct_disadvantaged_community',
    'pct_low_income_community',
    'acres',
    'geom']

In [ ]:
stats_joined_url = f's3://public-ca30x30/CA_Nature/2024/Preprocessing/v3/stats/*_habitat_climate_stats.parquet'
joined_stats = (con
                .read_parquet(stats_joined_url)
                .mutate(geom = _.geom.convert('epsg:3310','epsg:4326'))
                .rename(pct_grazing_lands = "pct_grazing")
                .mutate(gap_code = _.gap_code.substitute({'Non-Conservation Area':'None'}))
                .mutate(name = _.name.fill_null('None'))
                .mutate(manager = _.manager.fill_null('None'))
                .mutate(manager_type = _.manager_type.fill_null('None'))
                .mutate(gap_code = _.gap_code.fill_null('None'))
                .mutate(status = _.status.fill_null('None'))
                .mutate(land_tenure = _.land_tenure.fill_null('None'))
                .mutate(access_type = _.access_type.fill_null('None'))
                .mutate(county = _.county.fill_null('None'))
                .mutate(ecoregion = _.ecoregion.fill_null('None'))
                .mutate(habitat_type = _.habitat_type.fill_null('None'))
                .mutate(climate_zone = _.climate_zone.fill_null('None'))
)

url1 = f's3://public-ca30x30/CA_Nature/2024/Preprocessing/v3/ca30x30_habitat_climate_stats.parquet'
url2= f's3://public-ca30x30/ca30x30_cbn_v3.parquet'
data = joined_stats.select(cols).order_by('gap_code','county','name','id','sub_id')
data.to_parquet(url1)
data.to_parquet(url2)
